In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = "retina"
from matplotlib import rcParams
import cv2

rcParams["figure.figsize"]=(20,28)

img = plt.imread("../Image Cuts/TheWall.bmp")
plt.imshow(img)
print(np.shape(img))

In [ ]:
#split images in two seprate parts

imgL= img[:,:870]
imgR=img[:,870:]
print(np.shape(imgL))
print(np.shape(imgR))

fig, ax = plt.subplots(1,2)
ax[0].imshow(imgL)
ax[1].imshow(imgR)

In [ ]:
#overlapping the two images

overlap = 70
heightL, widthL, temp = imgL.shape
heightR, widthR, temp = imgR.shape

stitch = np.concatenate((imgL[:,:widthL - int(overlap/2)],imgR[:,int(overlap/2):]), axis = 1)
plt.imshow(stitch)

In [ ]:
rcParams["figure.figsize"] = 20,10
tiny_stitch = stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2, widthL-overlap*2:widthL+overlap]
plt.imshow(tiny_stitch)

In [ ]:
rcParams['figure.figsize'] = (20,8)

tinyL = imgL[:100,widthL-overlap:]
tinyR = imgR[:100,:overlap]

# display images
fig, ax = plt.subplots(1,2)
ax[0].imshow(tinyL)
ax[1].imshow(tinyR)

In [ ]:
rcParams['figure.figsize'] = (20,10)

# Turn both sections to grayscale
tinyL_g = np.dot(tinyL[...,:3], [0.299,0.587,0.114])
tinyR_g = np.dot(tinyR[...,:3], [0.299,0.587,0.114])

# Calculate their squared difference
diff = cv2.subtract(tinyL_g,tinyR_g)
diff = cv2.multiply(diff,diff)

# Display the squared difference
plt.imshow(diff,cmap='gray')
np.shape(diff)

In [ ]:
def get_seam(dif):
    height, width = dif.shape
    cost = dif.copy()
    seam = np.zeros((height))
    
    # Calculate the path costs by traversing the difference top down.
    # For every cost at level j, add the pixel value to the minimum cost from its 
    # 2 or 3 neighbors at lejel j-1 (above).
    for j in range(height):
        if(j == 0):
            continue
        for i in range(width):
            if(i == 0):
                cost[j,i] += np.amin((cost[j-1,i],cost[j-1,i+1]))
                continue
            if(i == width-1):
                cost[j,i] += np.amin((cost[j-1,i-1],cost[j-1,i]))
                continue
            cost[j,i] += np.amin((cost[j-1,i-1],cost[j-1,i],cost[j-1,i+1]))
    
    # Produce the seam by traversing the cost array, picking the lowest-cost
    # elements, following a continuous path.
    for j in reversed(range(height)):
        if(j == height-1):
            seam[j] = np.argmin((cost[j]))
            continue
        down = int(seam[j+1])
        if(down == 0):
            seam[j] = (down) + np.argmin((cost[j,down],cost[j,down+1]))
            continue
        if(down == width-1):
            seam[j] = (down-1) + np.argmin((cost[j,down-1],cost[j,down]))
            continue
        seam[j] = (down-1) + np.argmin((cost[j,down-1],cost[j,down],cost[j,down+1]))
    return seam

height, width = diff.shape
my_seam = get_seam(diff)

# Show the seam in the squared difference with white pixels
diff_seam = diff.copy()
height, width = diff.shape
for i in range(height):
    diff_seam[i,int(my_seam[i])] = 255 * 255 # This is white squared
plt.imshow(diff_seam,cmap='gray')
print(height)
print(width)

In [ ]:
rcParams['figure.figsize'] = (20,8)
tinyL = imgL[:,widthL-overlap:]
tinyR = imgR[:,:overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tinyL)
ax[1].imshow(tinyR)

In [ ]:
# Turn both sections to grayscale
tinyL_g = np.dot(tinyL[...,:3], [0.299,0.587,0.114])
tinyR_g = np.dot(tinyR[...,:3], [0.299,0.587,0.114])

# Calculate their squared difference
diff = cv2.subtract(tinyL_g,tinyR_g)
diff = cv2.multiply(diff,diff)

# Display the squared difference
plt.imshow(diff,cmap='gray')
np.shape(diff)

In [ ]:
rcParams['figure.figsize'] = (20,28)
height, width = diff.shape
my_seam = get_seam(diff)

# Show the seam in the squared difference with white pixels
diff_seam = diff.copy()
height, width = diff.shape
for i in range(height):
    diff_seam[i,int(my_seam[i])] = 255 * 255 # This is white squared
plt.imshow(diff_seam,cmap='gray')
print(height)
print(width)

In [ ]:
middle = tinyL.copy()    # Start with Left overlap.

# For every row in the middle section, replace Left pixels with 
# Right pixels starting at the seam.
for i in range(height):   
    j = int(my_seam[i])
    while j < width:
        middle[i,j] = tinyR[i,j]
        j += 1
    
# Now let's create a marked middle section to show the seam with red pixels.
middle_marked = middle.copy()
for i in range(height):
    middle_marked[i,int(my_seam[i])] = np.array((255,0,0))
        
rcParams['figure.figsize'] = (20,28)        
plt.imshow(middle_marked,cmap='gray')

In [ ]:
cut_stitch = np.concatenate((imgL[:,:widthL-overlap], middle_marked), axis=1)
cut_stitch = np.concatenate((cut_stitch, imgR[:,overlap:]), axis=1)
plt.imshow(cut_stitch)

In [ ]:

cut_stitch = np.concatenate((imgL[:,:widthL-overlap], middle), axis=1)
cut_stitch = np.concatenate((cut_stitch, imgR[:,overlap:]), axis=1)
plt.imshow(cut_stitch)

In [ ]:
rcParams['figure.figsize'] = (20,10)

tiny_stitch = stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]
tiny_cut_stitch = cut_stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tiny_stitch)
ax[1].imshow(tiny_cut_stitch)

In [ ]:
plt.imsave("stitched.bmp",cut_stitch)

In [ ]:
imgR = img[:,:870]
imgL = img[:,870:]

overlap = 70 
heightL, widthL, temp = imgL.shape
heightR, widthR, temp = imgR.shape

stitch = np.concatenate((imgL[:,:widthL-int(overlap/2)], imgR[:,int(overlap/2):]), axis=1)

# Left and righe overlap components.
tinyL = imgL[:,widthL-overlap:]
tinyR = imgR[:,:overlap]

# Now let's caluclate their difference.
# Turn both sections to grayscale
tinyL_g = np.dot(tinyL[...,:3], [0.299,0.587,0.114])
tinyR_g = np.dot(tinyR[...,:3], [0.299,0.587,0.114])

# Calculate their squared difference
diff = cv2.subtract(tinyL_g,tinyR_g)
diff = cv2.multiply(diff,diff)

# Let's calculate the seam for the squared difference
height, width = diff.shape
my_seam = get_seam(diff)


# First let's create the middle section.
middle = tinyL.copy()    # Start with Left overlap.
# For every row in the middle section, replace Left pixels with 
# Right pixels starting at the seam.
for i in range(height):   
    j = int(my_seam[i])
    while j < width:
        middle[i,j] = tinyR[i,j]
        j += 1
        
# Now let's create the stitched image.
cut_stitch = np.concatenate((imgL[:,:widthL-overlap], middle), axis=1)
cut_stitch = np.concatenate((cut_stitch, imgR[:,overlap:]), axis=1)
plt.imshow(cut_stitch)

In [ ]:
rcParams['figure.figsize'] = (20,10)

tiny_stitch = stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]
tiny_cut_stitch = cut_stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tiny_stitch)
ax[1].imshow(tiny_cut_stitch)

In [ ]:
tiny_stitch = stitch[750:,widthL-overlap*2:widthL+overlap]
tiny_cut_stitch = cut_stitch[750:,widthL-overlap*2:widthL+overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tiny_stitch)
ax[1].imshow(tiny_cut_stitch)